# Data Science com a linguagem Julia 

# Aprendizado não supervisionado para detecção de outliers

In [24]:
# Dados médicos de pacientes com diagnostico de câncer

In [25]:
# Importando de pacotes e instalando

import Pkg

Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("ScikitLearn")
Pkg.add("Clustering")
Pkg.add("Distances")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [26]:
using CSV, DataFrames, ScikitLearn, Clustering, Distances, Statistics

In [27]:
# carregando o conjunto de dados

dados = CSV.File(read("dataset.csv")) |> DataFrame

Row,tumorsize,co2,pain,wound,mobility,ntumors,nmorphine,remission,lungcapacity,Age,Married,FamilyHx,SmokingHx,Sex,CancerStage,LengthofStay,WBC,RBC,BMI,IL6,CRP,DID,Experience,School,Lawsuits,HID,Medicaid
,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Int64,String3,String7,String7,String3,Int64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,String7,Int64,Int64,Float64
1,67.9812,1.53433,4,4,2,0,0,0,0.801088,64.9682,0,no,former,male,II,6,6087.65,4.86842,24.1442,3.69898,8.08642,1,25,average,3,1,0.605867
2,64.7025,1.67613,2,3,2,0,0,0,0.326444,53.9171,0,no,former,female,II,6,6700.31,4.67905,29.4052,2.62748,0.803488,1,25,average,3,1,0.605867
3,51.567,1.53345,6,3,2,0,0,0,0.565031,53.3473,1,no,never,female,II,5,6042.81,5.00586,29.4826,13.8962,4.03416,1,25,average,3,1,0.605867
4,86.438,1.4533,3,3,2,0,0,0,0.848411,41.368,0,no,former,male,I,5,7162.7,5.26506,21.5573,3.00803,2.12586,1,25,average,3,1,0.605867
5,53.4002,1.56635,3,4,2,0,0,0,0.886491,46.8004,0,no,never,male,II,6,6443.44,4.98426,29.8152,3.8907,1.34932,1,25,average,3,1,0.605867
6,51.6573,1.41787,4,5,2,0,0,0,0.701031,51.9294,1,no,never,male,I,5,6800.55,5.19971,27.1025,1.41822,2.19469,1,25,average,3,1,0.605867
7,78.9171,1.70663,3,4,2,0,0,0,0.890854,53.8293,1,no,current,female,II,4,6236.45,4.8699,21.1191,2.29322,8.60796,1,25,average,3,1,0.605867
8,69.8332,1.52873,3,3,3,0,0,0,0.66088,46.5622,0,no,former,male,II,5,3997.7,4.64792,42.4754,4.57562,0.838388,1,25,average,3,1,0.605867
9,62.8526,1.54296,4,4,3,2,0,0,0.908871,54.3894,1,yes,former,male,II,6,6652.8,4.82706,18.6851,8.0302,7.22613,1,25,average,3,1,0.605867


In [28]:
# Dimensões do dataFrame
size(dados)

(8525, 27)

In [39]:
show(dados, allcols = true)

8525×27 DataFrame
  Row │ tumorsize  co2      pain   wound  mobility  ntumors  nmorphine  remission  lungcapacity  Age      Married  FamilyHx  SmokingHx  Sex      CancerStage  LengthofStay  WBC      RBC      BMI      IL6        CRP        DID    Experience  School   Lawsuits  HID    Medicaid 
      │ Float64    Float64  Int64  Int64  Int64     Int64    Int64      Int64      Float64       Float64  Int64    String3   String7    String7  String3      Int64         Float64  Float64  Float64  Float64    Float64    Int64  Int64       String7  Int64     Int64  Float64  
──────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    1 │   67.9812  1.53433      4      4         2        0          0          0      0.801088  64.9682        0  no        former     male     II       

In [29]:
# Visualizando os dados em todas as colunas

# show(dados, allcols = true)

show(dados)

8525×27 DataFrame
  Row │ tumorsize  co2      pain   wound  mobility  ntumors  nmorphine  remiss ⋯
      │ Float64    Float64  Int64  Int64  Int64     Int64    Int64      Int64  ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │   67.9812  1.53433      4      4         2        0          0         ⋯
    2 │   64.7025  1.67613      2      3         2        0          0
    3 │   51.567   1.53345      6      3         2        0          0
    4 │   86.438   1.4533       3      3         2        0          0
    5 │   53.4002  1.56635      3      4         2        0          0         ⋯
    6 │   51.6573  1.41787      4      5         2        0          0
    7 │   78.9171  1.70663      3      4         2        0          0
    8 │   69.8332  1.52873      3      3         3        0          0
    9 │   62.8526  1.54296      4      4         3        2          0         ⋯
   10 │   71.7779  1.59306      5      4         3        0          0

## Pré-Processamento de dados

In [30]:
# Importando do ScikitLearn a função para label encoding

In [37]:
@sk_import preprocessing: LabelEncoder

PyObject <class 'sklearn.preprocessing._label.LabelEncoder'>

In [38]:
# Criando o encoder

le = LabelEncoder() 

PyObject LabelEncoder()

In [40]:
# Label encoding das varáveis categóricas

In [42]:
dados[!, :FamilyHx] = le.fit_transform(dados[!, :FamilyHx])

dados[!, :SmokingHx] = le.fit_transform(dados[!, :SmokingHx])

dados[!, :Sex] = le.fit_transform(dados[!, :Sex])

dados[!, :CancerStage] = le.fit_transform(dados[!, :CancerStage])

dados[!, :School] = le.fit_transform(dados[!, :School])


8525-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [44]:
# Convertendo um dataframe em um array (matriz)

In [43]:
x = Matrix(dados)

8525×27 Matrix{Float64}:
 67.9812  1.53433  4.0  4.0  2.0  0.0  0.0  …  25.0  0.0  3.0   1.0  0.605867
 64.7025  1.67613  2.0  3.0  2.0  0.0  0.0     25.0  0.0  3.0   1.0  0.605867
 51.567   1.53345  6.0  3.0  2.0  0.0  0.0     25.0  0.0  3.0   1.0  0.605867
 86.438   1.4533   3.0  3.0  2.0  0.0  0.0     25.0  0.0  3.0   1.0  0.605867
 53.4002  1.56635  3.0  4.0  2.0  0.0  0.0     25.0  0.0  3.0   1.0  0.605867
 51.6573  1.41787  4.0  5.0  2.0  0.0  0.0  …  25.0  0.0  3.0   1.0  0.605867
 78.9171  1.70663  3.0  4.0  2.0  0.0  0.0     25.0  0.0  3.0   1.0  0.605867
 69.8332  1.52873  3.0  3.0  3.0  0.0  0.0     25.0  0.0  3.0   1.0  0.605867
 62.8526  1.54296  4.0  4.0  3.0  2.0  0.0     25.0  0.0  3.0   1.0  0.605867
 71.7779  1.59306  5.0  4.0  3.0  0.0  0.0     25.0  0.0  3.0   1.0  0.605867
 66.0256  1.60936  4.0  3.0  3.0  0.0  0.0  …  25.0  0.0  3.0   1.0  0.605867
 57.5208  1.6344   4.0  3.0  3.0  0.0  0.0     25.0  0.0  3.0   1.0  0.605867
 48.8394  1.53568  5.0  3.0  3.0  0.0  

In [47]:
# Pairwise para calcular a distância euclidiana  entre os pontos de dados

In [48]:
tipo_dist = Euclidean()
distancias = pairwise(tipo_dist, x', x')

8525×8525 Matrix{Float64}:
    0.0      612.842     50.725  1075.49   …   473.169  2186.07    532.313
  612.842      0.0      657.752   463.14       552.294  2790.32    489.015
   50.725    657.752      0.0    1120.58       497.574  2142.09    562.426
 1075.49     463.14    1120.58      0.0        929.533  3248.54    838.722
  356.665    257.23     400.839   720.086      423.912  2536.53    408.036
  713.244    101.213    757.857   364.022  …   624.504  2889.56    550.955
  149.682    464.227    196.177   926.388      417.871  2332.45    451.344
 2090.12    2702.66    2045.27   3165.11      2365.29    412.115  2466.32
  565.32      49.5982   610.236   510.681      521.594  2743.33    464.432
  310.332    922.325    265.828  1384.84       684.168  1882.88    769.033
   97.3158   518.393    140.718   980.894  …   432.804  2278.91    477.216
 1080.35    1692.78    1035.35   2155.36      1381.52   1142.92   1479.77
  370.008    244.105    414.26    707.068      427.882  2549.8     408.898


In [49]:
# Função para calcular o coeficiente Silhouette
function calcula_coef(sil)
    silMean = mean(sil)
    println("Média do coeficiente Silhouette $sil")
end

calcula_coef (generic function with 1 method)

In [50]:
# Função para avaliar o resultado da clusterização

In [51]:
function avalia_resultado(c)
    num = size(c)
    num = num[1]
    max = findmax(c)
    maxIndex = max[2]
    maxNum = max[1]
    min = findmin(c)
    minIndex = min[2]
    minNum = min[1]
    println("Número de clusteres: $num")
    println("Maior grupo: $maxNum no cluster maxIndex")
end

avalia_resultado (generic function with 1 method)

In [52]:
# Número de clusters

In [53]:
num_clusters = 30

30

# Clusterização com algoritmo K-means


In [54]:
# Criando o modelo

In [55]:
modelo = @time kmeans(distancias, num_clusters)

 95.079378 seconds (9.66 M allocations: 495.993 MiB, 0.10% gc time, 5.33% compilation time)


KmeansResult{Matrix{Float64}, Float64, Int64}([869.7218721433246 2233.0003814613387 … 1063.8568926610458 2679.608663743599; 1468.6998434912498 2842.9481956346276 … 483.2433970097921 3289.866076279968; … ; 1332.0927766113966 246.38120366370845 … 3194.6313927525257 566.2752113758612; 1202.131235744043 2573.814575492268 … 735.3757004508836 3017.6894259602896], [16, 27, 6, 29, 18, 22, 25, 2, 27, 4  …  6, 16, 3, 5, 3, 4, 24, 9, 2, 18], [3.801993443056488e7, 2.5751484285949707e7, 5.268814674600983e7, 3.247283527444458e7, 2.6512200610076904e7, 3.2265356601158142e7, 2.9335814121585846e7, 1.2939603219953918e8, 4.0627926112140656e7, 5.408740241665268e7  …  3.134686689016342e7, 3.798329360771561e7, 2.1672064548259735e7, 1.7633288785157776e8, 2.197894223480606e7, 3.629123366196823e7, 1.600937302456665e7, 2.8056222177871704e7, 3.929173736210632e7, 3.462857946572113e7], [429, 107, 290, 470, 254, 428, 13, 227, 349, 227  …  150, 316, 232, 297, 399, 467, 390, 459, 254, 66], [429, 107, 290, 471, 254, 43

In [56]:
# Extraindo o custo total

In [57]:
custo_total = modelo.totalcost
println("Custo total: $custo_total\n")

Custo total: 2.6670545151730646e11



In [ ]:
# Avaliando o modelo
calcula_coef(silhouettes(modelo, distancias))
println()
avalia_resultado(counts(modelo))

Média do coeficiente Silhouette [0.08849998540412507, 0.11976287639266536, 0.024909447544564367, 0.12948735937493483, 0.1835990747960099, 0.0790087616784424, 0.19875252782998687, 0.08208790773705232, -0.010271282095997813, -0.008350878309016818, 0.11087090175704495, 0.09249892492934386, 0.16142928469842355, 0.15874807289964332, 0.10974227358246669, 0.17860798898468289, 0.17220431237835165, 0.2883771918864124, 0.120767570556924, 0.1665283349423914, 0.0674214200705493, 0.15759820350571363, 0.07626225959811073, 0.3592669521800418, 0.26132171929725, 0.07903024275747028, 0.16742916724062262, 0.018026376802201427, 0.06587002725631985, 0.08042055688922256, 0.17127299752938674, 0.12911252071505896, 0.12157434266905343, 0.20243507983945463, 0.03612037148217051, 0.05126622491735977, -0.01548809664715567, 0.15135211788882375, 0.15384931294031, 0.08085444872970826, 0.3042314021127913, 0.0570324709569342, 0.13827118751267387, 0.08400906471163705, -0.01334655584458777, 0.09219591367143254, 0.1675372

64026781905, 0.34851701903706, 0.3209984892632757, -0.031006087959381867, 0.31413939703579274, 0.23712424522791875, 0.31915310874994995, 0.26309084751532263, 0.28064004415601096, 0.06762333774942608, 0.1884280843093099, 0.19245025134902316, 0.1444884935997257, 0.3046339037491641, 0.10568731905453344, 0.33122236270412375, 0.13243998552335412, 0.48636351199516126, 0.12205671298700149, 0.4582161316390748, 0.05599259226894804, 0.20374071610014, 0.1334940986681965, 0.5553268698747806, 0.19866922885591576, -0.03464965853975499, 0.23867248827518128, 0.2646719927776717, 0.3592026180852803, 0.35387531791065974, 0.20595976568454943, 0.09463259454492134, 0.28591438395974655, 0.2933583816933578, 0.22461789991081527, 0.28998237766143564, 0.2566091390192683, 0.1950942172599649, 0.36849450348686175, 0.1588509554948948, 0.2747578011332913, 0.15072937374166362, 0.22854175552176814, 0.07724392211925746, 0.11332970578271861, 0.1804599090145137, 0.15222401946485342, 0.21953224798678894, 0.2986414692949644

45, 0.32422630942925246, 0.4891249234808992, 0.20227473617312064, 0.14960243873537626, 0.30327776953563546, 0.19688927980465032, 0.1773735668280343, 0.32933464229636245, 0.13917235422737861, 0.1712501464645877, 0.08824463249448955, 0.1082595772088486, 0.2557764482923425, 0.2990798644764001, 0.28268770006040744, 0.4081081429526766, 0.16657999247891175, 0.034191631308669646, 0.23752108719040288, 0.09091684158846403, 0.21796399629055474, 0.3219698531027473, 0.23760615937794705, 0.18028168742918071, 0.24366494944733197, 0.1927992143199243, 0.2602232518820222, 0.15343634584485866, 0.21602565927927564, 0.19548137412127475, -0.012357307595663891, 0.15347834348411082, 0.3321899525823325, 0.06574353330560001, 0.25691469993351157, -0.029882050015568207, 0.13997493995005839, 0.24202355389397923, 0.15112827517776672, 0.19659789470483824, 0.030527455340238818, 0.2632599314795968, 0.18842865215796, 0.03732620898526395, 0.09189773584879601, 0.08729100109276222, 0.31456441237775246, 0.3521335404570120

In [60]:
# Extraindo os custos individuais
custos = modelo.costs

8525-element Vector{Float64}:
 3.801993443056488e7
 2.5751484285949707e7
 5.268814674600983e7
 3.247283527444458e7
 2.6512200610076904e7
 3.2265356601158142e7
 2.9335814121585846e7
 1.2939603219953918e8
 4.0627926112140656e7
 5.408740241665268e7
 4.567723379400253e7
 7.119181370055008e7
 2.6233767932746887e7
 ⋮
 3.9419996342033386e7
 8.690695887640381e7
 3.134686689016342e7
 3.798329360771561e7
 2.1672064548259735e7
 1.7633288785157776e8
 2.197894223480606e7
 3.629123366196823e7
 1.600937302456665e7
 2.8056222177871704e7
 3.929173736210632e7
 3.462857946572113e7

In [59]:
# Detectamos os outliers

In [61]:
for i = 1:5
    maxCost = findmax(custos)
    index = maxCost[2]
    println("\nOutlier encontrado na linha $index\n")
    println(dados[index, 1:10])
    custos[index] = 0
end
    


Outlier encontrado na linha 502

DataFrameRow
 Row │ tumorsize  co2      pain   wound  mobility  ntumors  nmorphine  remission  lungcapacity  Age     
     │ Float64    Float64  Int64  Int64  Int64     Int64    Int64      Int64      Float64       Float64 
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────
 502 │   84.0779  1.79721      6      4         3        0          5          0      0.684788  40.7941

Outlier encontrado na linha 8141

DataFrameRow
  Row │ tumorsize  co2      pain   wound  mobility  ntumors  nmorphine  remission  lungcapacity  Age     
      │ Float64    Float64  Int64  Int64  Int64     Int64    Int64      Int64      Float64       Float64 
──────┼──────────────────────────────────────────────────────────────────────────────────────────────────
 8141 │   73.9747  1.42127      5      5         9        4          1          1      0.748055  59.9453

Outlier encontrado na linha 1483

DataFrameRow
  Row │ tumorsi